In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import sendgrid
import os
from sendgrid import To, Mail, Email, Content
import asyncio

In [3]:
load_dotenv(override=True)

True

In [4]:
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("vimalpillai1234@gmail.com")
    to_email = To("vimalpillai47@gmail.com")
    content = Content("text/plain","This is a important test email")
    mail = Mail(from_email,to_email,"Test email",content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


In [5]:
instructions1 = """You are a sales agent working for ComplAI, 
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write professional, serious cold emails."""

instructions2 = """You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."""

instructions3 = """You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."""

In [6]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4.1-mini"
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model="gpt-4.1-mini"
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4.1-mini"
)

In [8]:
result = Runner.run_streamed(sales_agent1, "Write a cold sales email.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance with AI-Powered Automation

Hi [First Name],

I hope this message finds you well. I’m reaching out to introduce ComplAI, an AI-driven SaaS platform designed to streamline your SOC 2 compliance process and audit preparedness.

Managing SOC 2 requirements can be time-consuming and complex. ComplAI automates documentation, monitors controls continuously, and provides actionable insights to ensure you stay audit-ready with minimal effort.

Would you be open to a brief call next week to explore how ComplAI can reduce your compliance burden and safeguard your business?

Best regards,  
[Your Full Name]  
[Your Position]  
ComplAI  
[Your Contact Information]

In [9]:
message = "Write a cold sales email"
results = await asyncio.gather(
    Runner.run(sales_agent1, message),
    Runner.run(sales_agent2, message),
    Runner.run(sales_agent3, message)
)

outputs = [result.final_output for result in results]

for output in outputs:
    print(output, end="\n\n")

Subject: Simplify Your SOC 2 Compliance with AI-Powered Automation

Hi [First Name],

I’m reaching out to introduce ComplAI, a SaaS solution designed to make SOC 2 compliance and audit preparation efficient and stress-free. Our AI-driven platform automates compliance workflows, continuously monitors controls, and provides actionable insights—all in one centralized dashboard.

With ComplAI, your team can:

- Accelerate audit readiness by automating evidence collection  
- Identify and remediate gaps proactively  
- Reduce manual workload and human error  

If streamlining your SOC 2 compliance process sounds valuable, I’d be happy to share a brief demo tailored to your organization’s needs.

Are you available for a quick 15-minute call next week?

Best regards,  
[Your Full Name]  
[Your Title]  
ComplAI  
[Your Email] | [Your Phone Number]  
[Company Website]

Subject: SOC2 Compliance? Let AI Handle the Stress (You Just Sit Back)

Hey [First Name],

Ever feel like SOC2 audits are the c

In [10]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
        Imagine you are a customer and pick the one you are most likely to respond to. \
        Do not give an explanation; reply with the selected email only.",
    model="gpt-4.1-mini"
)

In [13]:
message = "Write a cold sales email."
with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )
    outputs = [output.final_output for output in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)
    best = await Runner.run(sales_picker, emails)
    print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: Streamline Your SOC 2 Compliance with AI-Powered Automation

Hi [First Name],

I hope this message finds you well. Navigating SOC 2 compliance and preparing for audits can be a complex, time-consuming process that distracts from your core business.

At ComplAI, we’ve developed an AI-powered SaaS platform that simplifies SOC 2 compliance management—automating documentation, monitoring controls, and generating audit-ready reports. Our solution helps teams reduce manual effort, minimize risk, and accelerate audit readiness.

I’d love to schedule a brief call to discuss how ComplAI can support [Company Name] in achieving smoother, faster SOC 2 compliance.

Are you available for a quick 15-minute conversation next week?

Best regards,  
[Your Full Name]  
[Your Job Title]  
ComplAI  
[Your Phone Number]  
[Your Email Address]  
[Company Website]


In [17]:
@function_tool
def send_email(body: str):
    """Send out an email with the given body to all the sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("vimalpillai1234@gmail.com")
    to_email = To("vimalpillai47@gmail.com")
    content = Content("text/plain",body)
    mail = Mail(from_email,to_email,"Sales email",content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [19]:
# send_email("Hello")

In [21]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1",tool_description="Write a cold sales email.")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023146541BC0>, strict_json_schema=True, is_enabled=True)

In [22]:
description = "Write a cold sales email."

tool1 = sales_agent1.as_tool(tool_name="sales_agent1",tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]
tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000231464463E0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023146446980>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'stri

In [23]:
instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

In [24]:
sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4.1-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

In [25]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."


In [26]:
subject_writer = Agent(name="Email subject writer", instructions=subject_instructions)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions)
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

In [28]:
from typing import Dict

In [29]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("vimalpillai1234@gmail.com")
    to_email = To("vimalpillai47@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [30]:
tools = [subject_tool, html_tool, send_html_email]

In [31]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000231465991C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000231465993A0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to 

In [32]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


In [33]:
emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4.1-mini",
    handoff_description="Convert an email to HTML and send it"
)

In [34]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000231464463E0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023146446980>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string

In [35]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


In [36]:
sales_manager = Agent(
    name= "Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4.1-mini"
)

In [37]:
message = "Send out a cold sales email addressed to Dear CEO from ALice"
with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)